# Preamble

In [ ]:
import nb_mypy
%load_ext nb_mypy

In [ ]:
import re
import copy
import math
import hashlib
import heapq
import numpy as np
from collections.abc import Callable
from itertools import product
from collections import defaultdict, deque

# Day 1: Historian Hysteria

In [ ]:
EXAMPLE = """\
3   4
4   3
2   5
1   3
3   9
3   3\
"""

In [ ]:
def parse_input(text: str) -> tuple[list[int], list[int]]:
    left = sorted(map(int, text.split()[::2]))
    right = sorted(map(int, text.split()[1::2]))
    return left, right

In [ ]:
example_input = parse_input(EXAMPLE)

In [ ]:
def total_distance(left: list[int], right: list[int]) -> int:
    return sum([abs(pair[0] - pair[1]) for pair in zip(left, right)])

In [ ]:
assert total_distance(*example_input) == 11

In [ ]:
with open("input/day01.txt", "r") as f:
    day01_input = parse_input(f.read())

In [ ]:
print(f"Day 1 - part 1 solution is: {total_distance(*day01_input)}")

In [ ]:
def similarity_score(left: list[int], right: list[int]) -> int:
    return sum([l * sum([1 for r in right if l == r]) for l in left])

In [ ]:
assert similarity_score(*example_input) == 31

In [ ]:
print(f"Day 2 - part 2 solution is: {similarity_score(*day01_input)}")

# Day 2: Red-Nosed Reports

In [ ]:
EXAMPLE = """\
7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9\
"""

In [ ]:
def parse_input(text: str) -> list[list[int]]:
    return [list(map(int, row.split())) for row in text.splitlines()]

In [ ]:
example_reports = parse_input(EXAMPLE)

In [ ]:
def safe_reports(reports: list[list[int]]) -> int:
    count = 0
    for report in reports:
        asc = None
        safe = True
        for i in range(len(report) - 1):
            diff = report[i + 1] - report[i]
            if abs(diff) < 1 or abs(diff) > 3:
                    safe = False
                    break;
            if asc is None:
                asc = diff >= 0
            else:
                if (asc and diff < 0) or (not asc and diff >= 0):
                    safe = False
                    break;
        if safe:
            count += 1
    return count

In [ ]:
assert safe_reports(example_reports) == 2

In [ ]:
with open("input/day02.txt", "r") as f:
    day02_input = parse_input(f.read())

In [ ]:
print(f"Day 2 - part 1 solution is: {safe_reports(day02_input)}")

In [ ]:
def safe_reports_with_tolerance(reports: list[list[int]]) -> int:
    count = 0
    for report in reports:
        partial_reports = []
        for i in range(len(report)):
            partial_reports.append(report[0:i] + report[i + 1:])
        if safe_reports(partial_reports) > 0:
            count += 1
    return count

In [ ]:
assert safe_reports_with_tolerance(example_reports) == 4

In [ ]:
print(f"Day 2 - part 2 solution is: {safe_reports_with_tolerance(day02_input)}")

# Day 3: Mull It Over

In [ ]:
EXAMPLE = "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"

In [ ]:
def parse_input(text: str) -> list[tuple[int, int]]:
    return list(map(lambda x: (int(x[0]), int(x[1])), re.findall(r"mul\((\d{1,3}),(\d{1,3})\)", text)))

In [ ]:
example_instructions = parse_input(EXAMPLE)

In [ ]:
def execute(instructions: list[tuple[int, int]]) -> list[int]:
    return [mul[0] * mul[1] for mul in instructions]

In [ ]:
assert sum(execute(example_instructions)) == 161

In [ ]:
with open("input/day03.txt", "r") as f:
    day03_input = parse_input(f.read())

In [ ]:
print(f"Day 3 - part 1 solution is: {sum(execute(day03_input))}")

In [ ]:
EXAMPLE2 = "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"

In [ ]:
def parse_input_part2(text: str) -> list[tuple[int, int]]:
    enable_mul = True
    instructions = []
    for instr in re.findall(r"do\(\)|don't\(\)|mul\(\d{1,3},\d{1,3}\)", text):
        if instr == "do()":
            enable_mul = True
        elif instr == "don't()":
            enable_mul = False
        elif enable_mul:
            m = re.match(r"mul\((\d{1,3}),(\d{1,3})\)", instr)
            if m is not None:
                instructions.append((int(m.group(1)), int(m.group(2))))
    return instructions

In [ ]:
example2_instructions = parse_input_part2(EXAMPLE2)

In [ ]:
assert sum(execute(example2_instructions)) == 48

In [ ]:
with open("input/day03.txt", "r") as f:
    day03_input = parse_input_part2(f.read())

In [ ]:
print(f"Day 3 - part 2 solution is: {sum(execute(day03_input))}")

# Day 4: Ceres Search

In [ ]:
EXAMPLE = """\
MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX\
"""

In [ ]:
def parse_input(text: str) -> list[list[str]]:
    return [[c for c in row] for row in text.splitlines()]

In [ ]:
word_search = parse_input(EXAMPLE)

In [ ]:
def count_word(haystack: list[list[str]], needle: str) -> int:
    count = 0
    directions = [(1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1)]
    needle_chars = [c for c in needle]
    for m, row in enumerate(haystack):
        for n, char in enumerate(row):
            if char == needle_chars[0]:
                for direction in directions:
                    found = True
                    for i, c in enumerate(needle_chars[1:]):
                        try:
                            nm, nn = m + (i + 1) * direction[0], n + (i + 1) * direction[1]
                            if (nm < 0 or nn < 0) or haystack[nm][nn] != c:
                                found = False
                                break
                        except IndexError:
                            found = False
                            break
                    if found:
                        count += 1
    return count

In [ ]:
assert count_word(word_search, "XMAS") == 18

In [ ]:
with open("input/day04.txt", "r") as f:
    day04_input = parse_input(f.read())

In [ ]:
print(f"Day 4 - part 1 solution is: {count_word(day04_input, 'XMAS')}")

In [ ]:
def count_x_mas(haystack: list[list[str]]) -> int:
    count = 0
    for m, row in enumerate(haystack[1: -1], 1):
        for n, char in enumerate(row[1: -1], 1):
            if char == "A":
                mas1 = {haystack[m - 1][n - 1], haystack[m + 1][n + 1]}
                mas2 = {haystack[m - 1][n + 1], haystack[m + 1][n - 1]}
                if mas1 == mas2 == set(["M", "S"]):
                    count += 1
    return count

In [ ]:
assert count_x_mas(word_search) == 9

In [ ]:
print(f"Day 4 - part 2 solution is: {count_x_mas(day04_input)}")

# Day 5: Print Queue

In [ ]:
EXAMPLE = """\
47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47\
"""

In [ ]:
def parse_input(text: str) -> tuple[list[tuple[int, int]], list[list[int]]]:
    rules = []
    updates = []
    for line in text.split():
        m = re.match(r"(\d+)\|(\d+)", line)
        if m:
            rules.append((int(m.group(1)), int(m.group(2))))
        else:
            updates.append([int(x) for x in line.split(",")])
    return rules, updates

In [ ]:
example_rules, example_updates = parse_input(EXAMPLE)

In [ ]:
def filter_incorrect_updates(updates: list[list[int]], rules: list[tuple[int, int]]) -> tuple[list[list[int]], list[list[int]]]:
    valid: list[list[int]] = []
    invalid: list[list[int]] = []
    for update in updates:
        is_valid = True
        for i, page in enumerate(update):
            for rule in rules:
                if rule[0] == page and rule[1] in update:
                    if i > update.index(rule[1]):
                        is_valid = False
        if is_valid:
            valid.append(update)
        else:
            invalid.append(update)
    return valid, invalid

In [ ]:
example_valid, example_invalid = filter_incorrect_updates(example_updates, example_rules)

In [ ]:
assert sum([x[len(x) // 2] for x in example_valid]) == 143

In [ ]:
with open("input/day05.txt", "r") as f:
    rules, updates = parse_input(f.read())

In [ ]:
valid, invalid = filter_incorrect_updates(updates, rules)

In [ ]:
print(f"Day 5 - part 1 solution is: {sum([x[len(x) // 2] for x in valid])}")

In [ ]:
def order_update(update: list[int], rules: list[tuple[int, int]]) -> list[int]:
    is_updated = True
    while is_updated:
        is_updated = False
        for rule in rules:
            if rule[0] in update and rule[1] in update and update.index(rule[0]) > update.index(rule[1]):
                update.insert(update.index(rule[1]), update.pop(update.index(rule[0])))
                is_updated = True
    return update

In [ ]:
assert sum([x[len(x) // 2] for x in [order_update(x, example_rules) for x in example_invalid]]) == 123

In [ ]:
print(f"Day 5 - part 2 solution is: {sum([x[len(x) // 2] for x in [order_update(x, rules) for x in invalid]])}")

# Day 6: Guard Gallivant

In [ ]:
LabMap = list[list[str]]
Position = tuple[int, int]
Direction = tuple[int, int]
PosDir = tuple[Position, Direction]

UP = (-1, -0)
DOWN = (1, 0)
LEFT = (0, -1)
RIGHT = (0, 1)
CLOCKWISE = [UP, RIGHT, DOWN, LEFT]

In [ ]:
EXAMPLE = """\
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...\
"""

In [ ]:
def parse_input(text: str) -> LabMap:
    return [[c for c in row] for row in text.splitlines()]

In [ ]:
example_map = parse_input(EXAMPLE)

In [ ]:
def initial_posdir(lab_map: LabMap) -> PosDir:
    for m, row in enumerate(lab_map):
        for n, cell in enumerate(row):
            if cell == "^":
                return ((m, n), UP)
    return ((0, 0), UP)

In [ ]:
assert initial_posdir(example_map) == ((6, 4), UP)

In [ ]:
class LoopError(Exception):
    pass

def patrol_path(lab_map: LabMap, initial_position: PosDir) -> set[PosDir]:
    current_posdir = initial_position
    path: set[PosDir] = {current_posdir}
    rown = len(lab_map)
    coln = len(lab_map[0])
    while True:
        next_position = (current_posdir[0][0] + current_posdir[1][0], current_posdir[0][1] + current_posdir[1][1])
        if not (next_position[0] >= 0 and next_position[0] < rown and next_position[1] >= 0 and next_position[1] < coln):
            break
        if lab_map[next_position[0]][next_position[1]] == "#":
            if current_posdir[1] == UP:
                current_posdir = (current_posdir[0], RIGHT)
            elif current_posdir[1] == RIGHT:
                current_posdir = (current_posdir[0], DOWN)
            elif current_posdir[1] == DOWN:
                current_posdir = (current_posdir[0], LEFT)
            else:
                current_posdir = (current_posdir[0], UP)
        else:
            current_posdir = (next_position, current_posdir[1])
        
        if current_posdir in path:
            raise LoopError()
        
        path.add(current_posdir)
    return path

In [ ]:
assert len(set([x[0] for x in patrol_path(example_map, initial_posdir(example_map))])) == 41

In [ ]:
with open("input/day06.txt", "r") as f:
    lab_map = parse_input(f.read())

In [ ]:
print(f"Day 6 - part 1 solution is: {len(set([x[0] for x in patrol_path(lab_map, initial_posdir(lab_map))]))}")

In [ ]:
def looping_obstructions(lab_map: LabMap, initial_position: PosDir) -> list[Position]:
    positions = []
    orig_map = copy.deepcopy(lab_map)
    previous_mn = None
    for m, row in enumerate(lab_map):
        for n, cell in enumerate(row):
            if cell not in ["^", "#"]:
                lab_map[m][n] = "#"
                if previous_mn is not None:
                    lab_map[previous_mn[0]][previous_mn[1]] = orig_map[previous_mn[0]][previous_mn[1]]
                previous_mn = (m, n)
                try:
                    patrol_path(lab_map, initial_position)
                except LoopError:
                    positions.append((m, n))
    return positions

In [ ]:
assert len(looping_obstructions(example_map, initial_posdir(example_map))) == 6

In [ ]:
print(f"Day 6 - part 2 solution is: {len(looping_obstructions(lab_map, initial_posdir(lab_map)))}")

# Day 7: Bridge Repair

In [ ]:
Equation = tuple[int, list[int]]
Operators = dict[str, Callable[[int, int], int]]

In [ ]:
EXAMPLE = """\
190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20\
"""

In [ ]:
def parse_input(text: str) -> list[Equation]:
    equations = []
    for row in text.splitlines():
        test, values = row.split(": ")
        test = int(test)
        values = list(map(int, values.split()))
        equations.append((test, values))
    return equations

In [ ]:
example_equations = parse_input(EXAMPLE)

In [ ]:
operators = { "+": int.__add__, "*": int.__mul__}

In [ ]:
def solve(equation: Equation, operators: Operators) -> list[tuple[str, ...]]:
    test, values = equation
    valid_positions = []
    for combination in product(operators.keys(), repeat=len(values) - 1):
        operator = operators[combination[0]]
        result = operator(*values[:2])
        for i in range(1, len(combination)):
            operator = operators[combination[i]]
            result = operator(result, values[i + 1])
        if result == test:
            valid_positions.append(combination)
    return valid_positions

In [ ]:
assert sum([e[0] for e in example_equations if solve(e, operators)]) == 3749

In [ ]:
with open("input/day07.txt", "r") as f:
    equations = parse_input(f.read())

In [ ]:
print(f"Day 7 - part 1 solution is: {sum([e[0] for e in equations if solve(e, operators)])}")

In [ ]:
operators["||"] = lambda x, y: int(f"{x}{y}")

In [ ]:
assert sum([e[0] for e in example_equations if solve(e, operators)]) == 11387

In [ ]:
print(f"Day 7 - part 2 solution is: {sum([e[0] for e in equations if solve(e, operators)])}")

# Day 8: Resonant Collinearity

In [ ]:
CityMap = list[list[str]]
Coord = tuple[int, int]

In [ ]:
EXAMPLE = """\
............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............\
"""

In [ ]:
def parse_input(text: str) -> CityMap:
    return [[c for c in row] for row in text.splitlines()]

In [ ]:
example_citymap = parse_input(EXAMPLE)

In [ ]:
def find_antinodes(citymap: CityMap, part_two=False) -> set[tuple[int, int]]:
    group_by_freq = defaultdict(list)
    for m, row in enumerate(citymap):
        for n, c in enumerate(row):
            if c == ".":
                continue
            group_by_freq[c].append((m, n))
    antinodes = set()
    for freq, positions in group_by_freq.items():
        for pos in positions:
            for other_pos in positions:
                if pos == other_pos:
                    continue
                for i in range(1, 50):
                    if i > 1 and not part_two:
                        break
                    m, n = pos
                    m1, n1 = other_pos
                    ma = m + (m - m1) * i
                    na = n + (n - n1) * i
                    if ma >= 0 and ma < len(citymap) and na >= 0 and na < len(citymap[0]):
                        antinodes.add((ma, na))
                    m, n = other_pos
                    m1, n1 = pos
                    ma = m + (m - m1) * i
                    na = n + (n - n1) * i
                    if ma >= 0 and ma < len(citymap) and na >= 0 and na < len(citymap[0]):
                        antinodes.add((ma, na))
                if part_two:
                    antinodes.add(pos)
                    antinodes.add(other_pos)
    return antinodes

In [ ]:
assert len(find_antinodes(example_citymap)) == 14

In [ ]:
with open("input/day08.txt", "r") as f:
    citymap = parse_input(f.read())

In [ ]:
print(f"Day 8 - part 1 solution is: {len(find_antinodes(citymap))}")

In [ ]:
assert len(find_antinodes(example_citymap, True)) == 34

In [ ]:
print(f"Day 8 - part 2 solution is: {len(find_antinodes(citymap, True))}")

# Day 9: Disk Fragmenter

In [ ]:
SparseRepr = list[str]

In [ ]:
EXAMPLE = "2333133121414131402"

In [ ]:
def parse_input(text: str) -> SparseRepr:
    dense = list(map(int, [c for c in text]))
    sparse = []
    pos = 0
    id = 0
    while pos < len(dense):
        sparse.extend([str(id) for _ in range(dense[pos])])
        if pos + 1 < len(dense):
            sparse.extend(["." for _ in range(dense[pos + 1])])
        pos += 2
        id += 1
    return sparse

In [ ]:
example_diskmap = parse_input(EXAMPLE)

In [ ]:
def block_defrag(diskmap: SparseRepr) -> SparseRepr:
    i, j = 0, len(diskmap) - 1
    defrag = copy.copy(diskmap)
    while i < j:
        if defrag[i] == ".":
            while i < j:
                swap = False
                if defrag[j] != ".":
                    defrag[i], defrag[j] = defrag[j], defrag[i]
                    swap = True
                j -= 1
                if swap:
                    break
        i += 1
    return defrag

In [ ]:
assert "".join(block_defrag(example_diskmap)) == "0099811188827773336446555566.............."

In [ ]:
def checksum(diskmap: SparseRepr) -> int:
    value = 0
    for i, c in enumerate(diskmap):
        if c != ".":
            value += i * int(c)
    return value

In [ ]:
assert checksum(block_defrag(example_diskmap)) == 1928

In [ ]:
with open("input/day09.txt", "r") as f:
    diskmap = parse_input(f.read())

In [ ]:
print(f"Day 9 - part 1 solution is: {checksum(block_defrag(diskmap))}")

In [ ]:
def file_defrag(diskmap: SparseRepr) -> SparseRepr:
    i, j = 0, len(diskmap) - 1
    defrag = copy.copy(diskmap)
    while i < j:
        if defrag[j] == ".":
            j -= 1
            continue
        file_id = defrag[j]
        file_len = 1
        for k in range(j - 1, -1, -1):
            if defrag[k] == file_id:
                file_len += 1
            else:
                break
        for k in range(i, j):
            swapped = False
            if defrag[k] == ".":
                if all([defrag[k + l] == "." for l in range(1, file_len)]):
                    swapped = True
                    for m in range(file_len):
                        defrag[j - m] = "."
                        defrag[k + m] = file_id
            if swapped:
                break
        j -= file_len
    return defrag

In [ ]:
assert "".join(file_defrag(example_diskmap)) == "00992111777.44.333....5555.6666.....8888.."

In [ ]:
assert checksum(file_defrag(example_diskmap)) == 2858

In [ ]:
print(f"Day 9 - part 2 solution is: {checksum(file_defrag(diskmap))}")

# Day 10: Hoof It

In [ ]:
TopographicMap = list[list[int]]
Position = tuple[int, int]

In [ ]:
EXAMPLE = """\
89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732\
"""

In [ ]:
def parse_input(text: str) -> TopographicMap:
    return [[int(c) for c in row] for row in text.splitlines()]

In [ ]:
example_map = parse_input(EXAMPLE)

In [ ]:
def find_trailheads(topographic_map: TopographicMap) -> list[Position]:
    positions = []
    for m, row in enumerate(topographic_map):
        for n, height in enumerate(row):
            if height == 0:
                positions.append((m, n))
    return positions

In [ ]:
trailheads = find_trailheads(example_map)

In [ ]:
assert len(trailheads) == 9

In [ ]:
def find_trails(topographic_map: TopographicMap, start: Position) -> list[list[Position]]:
    trails = list()
    frontier = [(start, [start])]

    while frontier:
        curpos, curtrail = frontier.pop()
        m, n = curpos
        curvalue = topographic_map[m][n]
        is_an_end = True
        for dm, dn in [(-1, 0), (0, 1), (1, 0), (0, -1)]:
            m1, n1 = m + dm, n + dn
            adjpos = (m1, n1)
            if m1 < 0 or m1 >= len(topographic_map) or n1 < 0 or n1 >= len(topographic_map[0]):
                continue
            adjvalue = topographic_map[m1][n1]
            if len(curtrail) > 1 and (m1, n1) == curtrail[-2]:
                continue
            if adjvalue == curvalue + 1:
                nexttrail = curtrail + [adjpos]
                frontier.append((adjpos, nexttrail))
                is_an_end = False
        
        if is_an_end:
                trails.append(curtrail)
    return trails

In [ ]:
def trailhead_score(topographic_map: TopographicMap, trailhead: Position, part_two=False) -> int:
    trails = find_trails(topographic_map, trailhead)
    if part_two:
        return len(set([str(t) for t in trails if len(t) == 10]))
    else:
        return len(set([t[-1] for t in trails if len(t) == 10]))

In [ ]:
assert sum([trailhead_score(example_map, t) for t in trailheads]) == 36

In [ ]:
with open("input/day10.txt", "r") as f:
    island_map = parse_input(f.read())

In [ ]:
island_trailheads = find_trailheads(island_map)

In [ ]:
print(f"Day 10 - part 1 solution is: {sum([trailhead_score(island_map, t) for t in island_trailheads])}")

In [ ]:
assert sum([trailhead_score(example_map, t, True) for t in trailheads]) == 81

In [ ]:
print(f"Day 10 - part 2 solution is: {sum([trailhead_score(island_map, t, True) for t in island_trailheads])}")

# Day 11: Plutonian Pebbles

In [ ]:
EXAMPLE_1 = "0 1 10 99 999"

In [ ]:
EXAMPLE_2 = "125 17"

In [ ]:
def parse_input(text: str) -> dict[int, int]:
    return {int(digits): 1 for digits in text.split(" ")}

In [ ]:
example_stones_1 = parse_input(EXAMPLE_1)

In [ ]:
def evolve_stone(stone: int) -> list[int]:
    digits = str(stone)
    if stone == 0:
        return [1]
    elif len(digits) % 2 == 0:
        return [int(digits[:len(digits) // 2]), int(digits[len(digits) // 2:])]
    else:
        return [stone * 2024]

In [ ]:
def evolve_stones(stones: dict[int, int], times: int = 1) -> dict[int, int]:
    if times == 0:
        return stones
    evolved: dict[int, int] = defaultdict(int)
    for stone, number in stones.items():
        for stone in evolve_stone(stone):
            evolved[stone] += number
    return evolve_stones(evolved, times - 1)

In [ ]:
def count_stones(stones: dict[int, int]) -> int:
    return sum(stones.values())

In [ ]:
assert evolve_stones(example_stones_1) == {1: 2, 2024: 1, 0: 1, 9: 2, 2021976: 1}

In [ ]:
example_stones_2 = parse_input(EXAMPLE_2)

In [ ]:
assert count_stones(evolve_stones(example_stones_2, 6)) == 22
assert count_stones(evolve_stones(example_stones_2, 25)) == 55312

In [ ]:
with open("input/day11.txt", "r") as f:
    stones = parse_input(f.read())

In [ ]:
print(f"Day 11 - part 1 solution is: {count_stones(evolve_stones(stones, 25))}")

In [ ]:
print(f"Day 11 - part 2 solution is: {count_stones(evolve_stones(stones, 75))}")

# Day 12: Garden Groups

In [ ]:
EXAMPLE_1 = """\
AAAA
BBCD
BBCC
EEEC\
"""

In [ ]:
EXAMPLE_2 = """\
OOOOO
OXOXO
OOOOO
OXOXO
OOOOO\
"""

In [ ]:
EXAMPLE_3 = """\
RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE\
"""

In [ ]:
GardenMap = list[list[str]]
Region = set[Position]

UP = (-1, -0)
DOWN = (1, 0)
LEFT = (0, -1)
RIGHT = (0, 1)
CLOCKWISE = [UP, RIGHT, DOWN, LEFT]

In [ ]:
def parse_input(text: str) -> GardenMap:
    return [[char for char in line] for line in text.split()]

In [ ]:
example_map_1 = parse_input(EXAMPLE_1)
example_map_2 = parse_input(EXAMPLE_2)
example_map_3 = parse_input(EXAMPLE_3)

In [ ]:
def flood_fill(garden_map: GardenMap, m: int, n: int, visited: Region) -> Region:
    filled: Region = set()

    def recurse(m, n):
        if (m, n) in visited:
            return
        visited.add((m, n))
        filled.add((m, n))
        for (md, nd) in CLOCKWISE:
            m1, n1 = m + md, n + nd
            if m1 >= 0 and n1 >= 0 and m1 < len(garden_map) and n1 < len(garden_map[0]) and \
                garden_map[m1][n1] == garden_map[m][n]:
                recurse(m1, n1)

    recurse(m, n)
    
    return filled

In [ ]:
def discover_regions(garden_map: GardenMap) -> list[Region]:
    visited: Region = set()
    discovered: list[Region] = []
    for m, row in enumerate(garden_map):
        for n, plant_type in enumerate(garden_map):
            region = flood_fill(garden_map, m, n, visited)
            if region:
                discovered.append(region)
    return discovered

In [ ]:
def region_area(region: Region) -> int:
    return len(region)

In [ ]:
def region_perimeter(region: Region) -> int:
    perimeter = 0
    for (m, n) in region:
        for (md, nd) in CLOCKWISE:
            m1, n1 = m + md, n + nd
            if (m1, n1) not in region:
                perimeter += 1
    return perimeter

In [ ]:
def region_sides(region: Region) -> int:
    CLOCKWISE = [(0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1)]
    sides = 0
    for (m, n) in region:
        for i in range(0, 8, 2):
            (md1, nd1) = CLOCKWISE[i % 8]
            (md2, nd2) = CLOCKWISE[(i + 1) % 8]
            (md3, nd3) = CLOCKWISE[(i + 2) % 8]
            m1, n1 = m + md1, n + nd1
            m2, n2 = m + md2, n + nd2
            m3, n3 = m + md3, n + nd3
            if ((m1, n1) not in region and (m3, n3) not in region) or ((m1, n1) in region and (m3, n3) in region and (m2, n2) not in region):
                sides += 1
    return sides

In [ ]:
def total_price(regions: list[Region], part_two: bool = False) -> int:
    return sum([region_area(region) * (region_sides(region) if part_two else region_perimeter(region)) for region in regions])

In [ ]:
assert total_price(discover_regions(example_map_1)) == 140
assert total_price(discover_regions(example_map_2)) == 772
assert total_price(discover_regions(example_map_3)) == 1930

In [ ]:
with open("input/day12.txt", "r") as f:
    garden_map = parse_input(f.read())

In [ ]:
print(f"Day 12 - part 1 solution is: {total_price(discover_regions(garden_map))}")

In [ ]:
assert total_price(discover_regions(example_map_1), True) == 80
assert total_price(discover_regions(example_map_2), True) == 436
assert total_price(discover_regions(example_map_3), True) == 1206

In [ ]:
print(f"Day 12 - part 2 solution is: {total_price(discover_regions(garden_map), True)}")

# Day 13: Claw Contraption

In [ ]:
EXAMPLE = """\
Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279\
"""

In [ ]:
Vec2 = tuple[int, int]

In [ ]:
class ClawMachine:

    def __init__(self, a: Vec2, b: Vec2, prize: Vec2) -> None:
        self._a = a
        self._b = b
        self._prize = prize

    def solution(self) -> int:
        for a_press in range(1, 101):
            for b_press in range(1, 101):
                if self._a[0] * a_press + self._b[0] * b_press == self._prize[0] and \
                   self._a[1] * a_press + self._b[1] * b_press == self._prize[1]:
                    cost = 3 * a_press + b_press
                    return cost
        return False

    def solution_numeric(self) -> int:
        a = np.array([[self._a[0], self._b[0]], [self._a[1], self._b[1]]])
        b = np.array([self._prize[0] + 10**13, self._prize[1] + 10**13])
        x = np.linalg.solve(a, b)
        if abs(round(x[0]) - x[0]) < 0.001 and abs(round(x[1]) - x[1]) < 0.001:
            return round(x[0]) * 3 + round(x[1])
        else:
            return False

In [ ]:
def parse_input(text: str) -> list[ClawMachine]:
    claw_machines = []
    for machine in text.split("\n\n"):
        m_button_a = re.search(r"Button A: X\+(\d+), Y\+(\d+)", machine, re.MULTILINE)
        m_button_b = re.search(r"Button B: X\+(\d+), Y\+(\d+)", machine, re.MULTILINE)
        m_prize = re.search(r"Prize: X=(\d+), Y=(\d+)", machine, re.MULTILINE)
        if m_button_a and m_button_b and m_prize:
            button_a = (int(m_button_a.group(1)), int(m_button_a.group(2)))
            button_b = (int(m_button_b.group(1)), int(m_button_b.group(2)))
            prize = (int(m_prize.group(1)), int(m_prize.group(2)))
            claw_machines.append(ClawMachine(button_a, button_b, prize))
        else:
            raise ValueError(machine)
    return claw_machines

In [ ]:
claw_machines_example_1 = parse_input(EXAMPLE)

In [ ]:
assert sum([m.solution() for m in claw_machines_example_1]) == 480

In [ ]:
with open("input/day13.txt", "r") as f:
    claw_machines = parse_input(f.read())

In [ ]:
print(f"Day 13 - part 1 solution is: {sum([m.solution() for m in claw_machines])}")

In [ ]:
print(f"Day 13 - part 2 solution is: {sum([m.solution_numeric() for m in claw_machines])}")

# Day 14: Restroom Redoubt

In [ ]:
EXAMPLE = """\
p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2


p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3\
"""

In [ ]:
Position = tuple[int, int]
Velocity = tuple[int, int]

In [ ]:
def parse_input(text: str) -> list[tuple[Position, Velocity]]:
    robots = []
    for line in text.split("\n"):
        m = re.search(r"p=([-\d]+),([-\d]+) v=([-\d]+),([-\d]+)", line)
        if m:
            pos = (int(m.group(1)), int(m.group(2)))
            vel = (int(m.group(3)), int(m.group(4)))
            robots.append((pos, vel))
    return robots

In [ ]:
robots_example = parse_input(EXAMPLE)

In [ ]:
def simulate_positions(robots: list[tuple[Position, Velocity]], area: tuple[int, int], n: int = 1) -> list[list[Position]]:
    positions: list[Position] = [r[0] for r in robots]
    velocities = [r[1] for r in robots]
    result = []
    for _ in range(n):
        for i in range(len(positions)):
            positions[i] = (
                (positions[i][0] + velocities[i][0]) % area[0],
                (positions[i][1] + velocities[i][1]) % area[1],
            )
        result.append(copy.copy(positions))
    return result

In [ ]:
def draw_area(area: tuple[int, int], positions) -> None:
    for y in range(area[1]):
        for x in range(area[0]):
            count = 0
            for (px, py) in positions:
                if px == x and py == y:
                    count += 1
            if count > 0:
                print(f"{count}", end="")
            else:
                print(".", end="")
        print()

In [ ]:
draw_area((11, 7), simulate_positions(robots_example, (11, 7), 100)[-1])

In [ ]:
def safety_factor(area: tuple[int, int], positions) -> int:
    q1, q2, q3, q4 = 0, 0, 0, 0
    for (x, y) in positions:
        if x < area[0] // 2:
            if y < area[1] // 2:
                q2 += 1
            elif y > area[1] // 2:
                q3 += 1
        elif x > area[0] // 2:
            if y < area[1] // 2:
                q1 += 1
            elif y > area[1] // 2:
                q4 += 1
    return q1 * q2 * q3 * q4

In [ ]:
assert safety_factor((11, 7), simulate_positions(robots_example, (11, 7), 100)[-1]) == 12

In [ ]:
with open("input/day14.txt", "r") as f:
    robots = parse_input(f.read())

In [ ]:
print(f"Day 14 - part 1 solution is: {safety_factor((101, 103), simulate_positions(robots, (101, 103), 100)[-1])}")

In [ ]:
def avg_sqr_dist_from(positions: list[Position], pos: tuple[int, int]) -> float:
    distances = []
    for position in positions:
        distance = math.pow(position[0] - pos[0], 2) + math.pow(position[1] - pos[1], 2)
        distances.append(distance)
    return sum(distances) / len(distances)

In [ ]:
min_i, min_dist, min_positions = None, 100000.0, None
for (i, positions_at_i) in enumerate(simulate_positions(robots, (101, 103), 10000), 1):
    dist = avg_sqr_dist_from(positions_at_i, (50, 51))
    if dist < min_dist:
        min_i = i
        min_dist = dist
        min_positions = positions_at_i
print(f"{min_i} - {min_dist}")
draw_area((101, 103), min_positions)

# Day 15: Warehouse Woes

In [ ]:
SMALLER_EXAMPLE = """\
########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<^^>>>vv<v>>v<<\
"""

In [ ]:
LARGER_EXAMPLE = """\
##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^\
"""

In [ ]:
Grid = list[list[str]]
Position = tuple[int, int]
Directions = list[str]
ROBOT = "@"
BOX = "O"
WALL = "#"
EMPTY = "."
BOX_WIDE_L = "["
BOX_WIDE_R = "]"
UP = "^"
RIGHT = ">"
DOWN = "v"
LEFT = "<"
DIRECTIONS = {
    UP: (-1, 0),
    RIGHT: (0, 1),
    DOWN: (1, 0),
    LEFT: (0, -1),
}

In [ ]:
def parse_input(text: str) -> tuple[Grid, Directions]:
    grid_input, dirs_input = text.split("\n\n")
    grid = [[c for c in l] for l in grid_input.split("\n")]
    dirs = [c for c in dirs_input if c in DIRECTIONS.keys()]
    return grid, dirs

In [ ]:
grid_example_s, dirs_example_s = parse_input(SMALLER_EXAMPLE)
grid_example_l, dirs_example_l = parse_input(LARGER_EXAMPLE)

In [ ]:
def robot_position(grid: Grid) -> Position:
    for m, line in enumerate(grid):
        for n, c in enumerate(line):
            if c == ROBOT:
                return m, n
    raise ValueError()

In [ ]:
assert robot_position(grid_example_s) == (2, 2)
assert robot_position(grid_example_l) == (4, 4)

In [ ]:
def eval_move(grid: Grid, pos: list[Position], dir: str) -> list[Position]|None:
    nextpos = list()
    for m, n in pos:
        m1, n1 = m + DIRECTIONS[dir][0], n + DIRECTIONS[dir][1]
        if (m1, n1) not in nextpos:
            nextpos.append((m1, n1))
        if dir in [UP, DOWN]:
            if grid[m1][n1] == BOX_WIDE_L and (m1, n1 + 1) not in nextpos:
                nextpos.append((m1, n1 + 1))
            if grid[m1][n1] == BOX_WIDE_R and (m1, n1 - 1) not in nextpos:
                nextpos.append((m1, n1 - 1))

    if all([grid[m][n] == EMPTY for m, n in nextpos]):
        return pos
    elif any([grid[m][n] == WALL for m, n in nextpos]):
        return None
    else:
        next_moves = eval_move(grid, [(m, n) for m, n in nextpos if grid[m][n] in [BOX, BOX_WIDE_L, BOX_WIDE_R]], dir)
        if next_moves is None:
            return None
        else:
            return pos + next_moves

In [ ]:
assert eval_move(grid_example_s, [(1, 1)], UP) == None
assert eval_move(grid_example_s, [(1, 1)], LEFT) == None
assert eval_move(grid_example_s, [(1, 1)], RIGHT) == [(1, 1)]
assert eval_move(grid_example_s, [(1, 1)], DOWN) == None
assert eval_move(grid_example_s, [(1, 2)], UP) == None
assert eval_move(grid_example_s, [(1, 2)], LEFT) == [(1, 2)]
assert eval_move(grid_example_s, [(1, 2)], RIGHT) == [(1, 2), (1, 3)]

In [ ]:
def apply_robot_moves(start_grid: Grid, dirs: Directions) -> Grid:
    grid = copy.deepcopy(start_grid)
    robot_pos = robot_position(grid)
    for dir in dirs:
        moves = eval_move(grid, [robot_pos], dir)
        if moves is None:
            continue
        for i, (m, n) in enumerate(reversed(moves)):
            m1, n1 = m + DIRECTIONS[dir][0], n + DIRECTIONS[dir][1]
            grid[m1][n1] = grid[m][n]
            grid[m][n] = EMPTY
            if i == len(moves) - 1:
                robot_pos = (m1, n1)
    return grid

In [ ]:
result = apply_robot_moves(grid_example_s, dirs_example_s)

In [ ]:
assert "\n".join(["".join([c for c in l]) for l in result]) == """\
########
#....OO#
##.....#
#.....O#
#.#O@..#
#...O..#
#...O..#
########\
"""

In [ ]:
def find_boxes(grid: Grid) -> list[Position]:
    result = []
    for m, line in enumerate(grid):
        for n, c in enumerate(line):
            if c in [BOX, BOX_WIDE_L]:
                result.append((m, n))
    return result

In [ ]:
def coordinate(box: Position) -> int:
    return box[0] * 100 + box[1]

In [ ]:
assert sum([coordinate(p) for p in find_boxes(result)]) == 2028

In [ ]:
result = apply_robot_moves(grid_example_l, dirs_example_l)

In [ ]:
assert "\n".join(["".join([c for c in l]) for l in result]) == """\
##########
#.O.O.OOO#
#........#
#OO......#
#OO@.....#
#O#.....O#
#O.....OO#
#O.....OO#
#OO....OO#
##########\
"""

In [ ]:
assert sum([coordinate(p) for p in find_boxes(result)]) == 10092

In [ ]:
with open("input/day15.txt", "r") as f:
    warehouse, movements = parse_input(f.read())

In [ ]:
result = apply_robot_moves(warehouse, movements)

In [ ]:
print(f"Day 15 - part 1 solution is: {sum([coordinate(p) for p in find_boxes(result)])}")

In [ ]:
def wider(warehouse: Grid) -> Grid:
    result = []
    for row in warehouse:
        wrow = []
        for c in row:
            if c == "#":
                wrow.append("#")
                wrow.append("#")
            elif c == "O":
                wrow.append("[")
                wrow.append("]")
            elif c == ".":
                wrow.append(".")
                wrow.append(".")
            else:
                wrow.append("@")
                wrow.append(".")
        result.append(wrow)
    return result

In [ ]:
PART2_EXAMPLE = """\
#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######

<vv<<^^<<^^\
"""

In [ ]:
grid_example_part2, dirs_example_part2 = parse_input(PART2_EXAMPLE)

In [ ]:
grid_example_part2_wider = wider(grid_example_part2)

In [ ]:
assert "\n".join(["".join([c for c in l]) for l in grid_example_part2_wider]) == """\
##############
##......##..##
##..........##
##....[][]@.##
##....[]....##
##..........##
##############\
"""

In [ ]:
result = apply_robot_moves(grid_example_part2_wider, dirs_example_part2)

In [ ]:
assert "\n".join(["".join([c for c in l]) for l in result]) == """\
##############
##...[].##..##
##...@.[]...##
##....[]....##
##..........##
##..........##
##############\
"""

In [ ]:
grid_example_l_wider = wider(grid_example_l)

In [ ]:
assert "\n".join(["".join([c for c in l]) for l in grid_example_l_wider]) == """\
####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##....[]@.....[]..##
##[]##....[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################\
"""

In [ ]:
result = apply_robot_moves(grid_example_l_wider, dirs_example_l)

In [ ]:
assert "\n".join(["".join([c for c in l]) for l in result]) == """\
####################
##[].......[].[][]##
##[]...........[].##
##[]........[][][]##
##[]......[]....[]##
##..##......[]....##
##..[]............##
##..@......[].[][]##
##......[][]..[]..##
####################\
"""

In [ ]:
result = apply_robot_moves(wider(warehouse), movements)

In [ ]:
print(f"Day 15 - part 2 solution is: {sum([coordinate(p) for p in find_boxes(result)])}")

# Day 16: Reindeer Maze

In [ ]:
EXAMPLE_1 = """\
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############\
"""

In [ ]:
EXAMPLE_2 = """\
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################\
"""

In [ ]:
Pos = tuple[int, int]     # The position of a reindeer represented by m,n coordinates (topleft corner is 0,0)
PosDir = tuple[Pos, str]  # Both the position and the direction the reindeer is facing

In [ ]:
DIRECTIONS = {                 # Allowed movement directions and their relative offsets in m,n coordinates
    "N": (-1, 0),
    "E": (0, 1),
    "S": (1, 0),
    "W": (0, -1),
}

In [ ]:
class Maze:

    _TILE_START = "S"
    _TILE_END = "E"
    _TILE_WALL = "#"

    def __init__(self, puzzle_input: str):
        self._tiles = [[t for t in l] for l in puzzle_input.split("\n")]
        self._shape = (len(self._tiles), len(self._tiles[0]))
        self._start = self._find_tile(self._TILE_START)
        self._end = self._find_tile(self._TILE_END)

    def _find_tile(self, target: str) -> Pos:
        for m, line in enumerate(self._tiles):
            for n, tile in enumerate(line):
                if tile == target:
                    return m, n
        raise ValueError()

    def is_inside(self, m: int, n: int) -> bool:
        return m >= 0 and n >= 0 and m < len(self._tiles) and n < len(self._tiles[0])
    
    def is_wall(self, m: int, n: int) -> bool:
        return self.is_inside(m, n) and self._tiles[m][n] == self._TILE_WALL

    @property
    def shape(self) -> tuple[int, int]:
        return self._shape

    @property
    def tiles(self) -> list[list[str]]:
        return self._tiles

    @property
    def start(self) -> Pos:
        return self._start

    @property
    def end(self) -> Pos:
        return self._end

In [ ]:
class Path:

    _steps: list[PosDir]

    def __init__(self, maze: Maze, steps: list[PosDir] = []):
        self._maze = maze
        self._steps = [s for s in steps]

    def add_step(self, posdir: PosDir) -> None:
        self._steps.append(posdir)

    @property
    def steps(self) -> list[PosDir]:
        return self._steps

    def score(self) -> int:
        points = 1
        for i in range(1, len(self._steps)):
            _, pdir = self._steps[i - 1]
            _, cdir = self._steps[i]
            if pdir != cdir:
                points += 1000
            points += 1
        return points

    def digest(self) -> str:
        return hashlib.md5(str(self._steps).encode()).hexdigest()

    def __str__(self) -> str:
        ascii_map = ""
        tiles = copy.deepcopy(self._maze.tiles)
        for (m, n), dir in self._steps:
            tiles[m][n] = "^" if dir == "N" else ">" if dir == "E" else "v" if dir == "S" else "<"
        for line in tiles:
            ascii_map += "".join(line) + "\n"
        return f"Score {self.score()} - ID {self.digest()[:8]}\n{ascii_map}"

In [ ]:
maze_example_1 = Maze(EXAMPLE_1)
maze_example_2 = Maze(EXAMPLE_2)

In [ ]:
assert maze_example_1.start == (13, 1)
assert maze_example_2.start == (15, 1)

In [ ]:
def next_directions(cur_dir: str) -> list[str]:
    if cur_dir == "N":
        return ["W", "N", "E"]
    elif cur_dir == "E":
        return ["N", "E", "S"]
    elif cur_dir == "S":
        return ["E", "S", "W"]
    else:
        return ["S", "W", "N"]

In [ ]:
def find_paths(maze: Maze) -> list[Path]:
    paths = []
    visited: set[tuple[Pos, str]] = set()
    frontier: list[tuple[int, int, Pos, str, Path]] = [(0, 0, maze.start, "E", Path(maze))]
    counter = 0

    while frontier:
        score, _, (m, n), dir, path = heapq.heappop(frontier)
        if (m, n) == maze.end:
            paths.append(path)
            continue

        visited.add(((m, n), dir))
        path.add_step(((m, n), dir))

        for next_dir in next_directions(dir):
            md, nd = DIRECTIONS[next_dir]
            m1, n1 = m + md, n + nd
            next_score = score + (1 if dir == next_dir else 1000)
            counter += 1
            if maze.is_inside(m1, n1) and not maze.is_wall(m1, n1) and ((m1, n1), next_dir) not in visited:
                heapq.heappush(frontier, (next_score, counter, (m1, n1), next_dir, Path(maze, path.steps)))
    return paths

In [ ]:
paths_example_1 = find_paths(maze_example_1)
paths_example_2 = find_paths(maze_example_2)

In [ ]:
for path in paths_example_1:
    print(path)

In [ ]:
scores_example_1 = [p.score() for p in paths_example_1]
scores_example_2 = [p.score() for p in paths_example_2]

In [ ]:
best_score_example_1 = sorted(scores_example_1)[0]
best_score_example_2 = sorted(scores_example_2)[0]

In [ ]:
assert best_score_example_1 == 7036
assert best_score_example_2 == 11048

In [ ]:
with open("input/day16.txt", "r") as f:
    maze = Maze(f.read())

In [ ]:
paths = find_paths(maze)

In [ ]:
scores = [p.score() for p in paths]

In [ ]:
best_score = sorted(scores)[0]

In [ ]:
print(f"Day 16 - part 1 solution is: {best_score}")

In [ ]:
best_paths_example_1 = [p for p in paths_example_1 if p.score() == best_score_example_1]
best_paths_example_2 = [p for p in paths_example_2 if p.score() == best_score_example_2]

In [ ]:
best_spots_example_1 = set()
for path in best_paths_example_1:
    for pos, dir in path.steps:
        best_spots_example_1.add(pos)

In [ ]:
best_spots_example_2 = set()
for path in best_paths_example_2:
    for pos, dir in path.steps:
        best_spots_example_2.add(pos)

In [ ]:
assert len(best_spots_example_1) + 1 == 45
assert len(best_spots_example_2) + 1 == 64

In [ ]:
best_paths = [p for p in paths if p.score() == best_score]

In [ ]:
best_spots = set()
for path in best_paths:
    for pos, dir in path.steps:
        best_spots.add(pos)

In [ ]:
print(f"Day 16 - part 2 solution is: {len(best_spots) + 1}")